# Aim of the notebook
This notebook is used to merge the output of segmentation and present basic representation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import seaborn as sns

In [5]:
samples=['583B1_TLN','588B1_TLN','588B1_LNG3','588B2_LNG1','689C_LNG2','689C_LNG5','689C_LNG3','689C_LNG1',
        '640C_LNG5','640C_LNG3','640C_LNG2','583B_LNG2','588B_LNG2','583B_LNG5','583B_LNG3','588B_LNG6']

we read all individual h5ads we obtained from baysor segmentation

In [6]:
for sample in np.unique(samples):
    adata2=sc.read('F:/ISS/adata_baysor/'+sample+'_adata.h5ad')
    adata2.obs['sample']=sample
    adata2.obs['region']=sample.split('_')[1]
    try:
        adata=sc.concat([adata,adata2],join='outer')
    except:
        adata=adata2

C:\Users\sergio.salas\Miniconda3\lib\site-packages\anndata\_core\anndata.py:119: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation na

We can save the object, which will be used as input in the same step,  and read again if needed

In [ ]:
adata.write('F:/ISS/adata_baysor/adata_16samples_unprocessed.h5ad')
adata=sc.read('F:/ISS/adata_baysor/adata_16samples_unprocessed.h5ad')

... storing 'sample' as categorical
... storing 'region' as categorical


In [ ]:
maximum_numbers_of_reads=100
minimum_number_of_reads=3

adata.obs['total_counts']=list(np.sum(adata.X,axis=1))
adata=adata[adata.obs['total_counts']>minimum_number_of_reads] #adjust the numbers to keep good cells according to appropiate ones
adata=adata[adata.obs['total_counts']<maximum_numbers_of_reads] #adjust the numbers to more than

 We filter cells with low number of genes expressed and normalize cells

In [ ]:
sc.pp.filter_cells(adata,min_genes=3)
adata.raw=adata
sc.pp.normalize_total(adata, target_sum=100)
sc.pp.log1p(adata)

In [78]:
sc.tl.pca(adata, svd_solver='arpack')
plt.rcParams['figure.facecolor'] = 'white'
sc.pl.pca_variance_ratio(adata, log=False)

We identify expression-based neighbors for each cell and perform umap and clustering

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=0)
sc.tl.leiden(adata,resolution=1.0)
sc.tl.umap(adata,min_dist=0.0005)

## Next we represent umaps, colored by several metadata

In [ ]:
plt.rcParams['figure.facecolor'] = 'white'
sc.set_figure_params(figsize=(14,10),dpi=600)
sc.pl.umap(adata,color=['leiden'],cmap='hsv',s=30)#,'total_counts'])#,save='chickbrain_UMAP_bins.pdf')
plt.rcParams['figure.facecolor'] = 'white'
sc.set_figure_params(figsize=(14,10),dpi=600)
sc.pl.umap(adata,color=['region'],cmap='hsv',s=30)#,'total_counts'])#,save='chickbrain_UMAP_bins.pdf')
plt.rcParams['figure.facecolor'] = 'white'
sc.set_figure_params(figsize=(14,10),dpi=600)
sc.pl.umap(adata,color=['sample'],cmap='hsv',s=30)#,'total_counts'])#,save='chickbrain_UMAP_bins.pdf')

In [ ]:
for va in adata.var.index:
    plt.rcParams['figure.facecolor'] = 'white'
    sc.set_figure_params(figsize=(15,15),dpi=50)
    sc.pl.umap(adata,color=[va],cmap='hsv',s=10)#,'total_counts'])#,save='chickbrain_UMAP_bins.pdf')

In [25]:
adata.obs['total_counts']=list(np.sum(adata.X,axis=1))

We identify Differentially expressed genes

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon')
sc.pl.rank_genes_groups(adata, n_genes=5, sharey=False)

This is a function to generate spatial maps of the cells, colored by cell type

In [47]:
def map_of_clusters(adata,key='leiden',clusters='all',size=8,background='white',figuresize=(10,7),save=None,format='pdf'):
    
    #PARAMETERS TO MODIFY:
    #-key: the terms in adata.obs that you want to plot
    #-clusters:'all' for plotting all clusters in a single plot, 'individual': for plots of individual genes, or ['3','5'] (your groups
    #          between square brackets to plot only some clusters
    #-size: to change the size of your spots
    #-background: to change the color of the background
    #-figuresize: to specify the size of your figure
    #-save: if you want to save your figure, give the PATH of the folder where you want to save it
    #-format: specify the format in which you want to save your figure
    
    try:
        adata.obs[key]=adata.obs[key].astype(int)
        colors=dict(zip(np.unique(adata.obs[key]),adata.uns[key+'_colors']))
    except:
        colors=dict(zip(np.unique(adata.obs[key]),adata.uns[key+'_colors']))
    #cl.apply(lambda x: colors[x])
    plt.rcParams['figure.facecolor'] = background
    if clusters=='all':
        cl=adata.obs[key]
        plt.figure(figsize=figuresize)
        figa=plt.scatter(x=adata.obs.X,y=adata.obs.Y,c=cl.apply(lambda x: colors[x]),s=size,linewidths=0, edgecolors=None)
        plt.axis('off')
        if not save==None:
            plt.savefig(save +'/map_all_clusters_'+str(size)+'_'+background+'_'+key+'.'+format)
    elif clusters=='individual':
        cl=adata.obs[key]
        for each in adata.obs[key].unique():
            adatasub=adata[adata.obs[key]==each]
            plt.figure(figsize=figuresize)
            plt.scatter(x=adata.obs.X,y=adata.obs.Y,c='grey',s=size/5,linewidths=0, edgecolors=None)
            cl=adatasub.obs[key]
            plt.scatter(x=adatasub.obs.X,y=adatasub.obs.Y,c=cl.apply(lambda x: colors[x]),s=size,linewidths=0, edgecolors=None)
            plt.axis('off')
            plt.title('Group: '+ str(each))
            if not save==None:
                plt.savefig(save +'/map_inidivdual_cluster_'+str(each)+'_'+str(size)+background+'_'+key+'.'+format)
    else:
        adatasub=adata[adata.obs[key].isin(clusters)]
        plt.figure(figsize=figuresize)
        plt.scatter(x=adata.obs.X,y=adata.obs.Y,c='grey',s=size/5,linewidths=0, edgecolors=None)
        cl=adatasub.obs[key]
        plt.scatter(x=adatasub.obs.X,y=adatasub.obs.Y,c=cl.apply(lambda x: colors[x]),s=size,linewidths=0, edgecolors=None)
        plt.axis('off')
        plt.legend()
        if not save==None:
                s=''
                for element in clusters:
                    s=s+str(element)
                print(s)
                plt.savefig(save +'/map_group_of_clusters_'+str(s)+'_'+str(size)+background+'_'+key+'.'+format)
#        plt.title('Group: '+ paste(clusters))

In [ ]:
adata.obs['X']=adata.obs['x']
adata.obs['Y']=adata.obs['y']
for a in adata.obs['sample'].unique():
    ads=adata[adata.obs['sample']==a]
    sc.pl.umap(ads,color=['leiden'],cmap='hsv',s=30)#,'total_counts'])#,save='chickbrain_UMAP_bins.pdf')
    map_of_clusters(ads,figuresize=(10,10),size=5,clusters='all')